**Step-1: Install the packages**

In [ ]:
! pip install crewai
! pip install crewai-tools

**Step-2: Import the required packages**

In [ ]:
import os
from crewai import Agent, Task, Crew, Process, LLM  #crew Ai packages
from crewai_tools import SerperDevTool  # serper tool

**Step-3: Access the keys**

In [ ]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Get API keys
gemini_key = os.getenv("GEMINI_KEY")
serper_key = os.getenv("SERPER_KEY")

# Set them as environment variables
os.environ["GOOGLE_API_KEY"] = gemini_key
os.environ["SERPER_API_KEY"] = serper_key

# Optional: Print to check if keys are loaded correctly (Avoid printing sensitive info in public)
# print(gemini_key)
# print(serper_key)


**Step-4:Access the LLM and Tool**

In [ ]:
# Initialize a search tool (to fetch real-time travel info)
search_tool = SerperDevTool()

# Define the AI Model
llm = LLM(model="gemini/gemini-1.5-flash",
          verbose=True, # I want to see the statements
          temperature=0.5, # thinking power
          api_key=os.environ["GOOGLE_API_KEY"])

**Step-5: Intialize the agents**

In [ ]:
# 🧭 Travel Researcher Agent (Finds historical sites + weather)
researcher = Agent(
    role="Travel Researcher",
    goal="Find most popular sites, public transport hotels, and real-time weather for {destination}.",
    verbose=True, # when I run the code I want to see the statements
    memory=True, # store
    backstory="You are an expert travel researcher, providing up-to-date information about history-focused trips.",
    llm=llm,
    tools=[search_tool],  # Uses live search tool
    allow_delegation=True # transfer the task to the next agent
)

# 💰 Budget Planner Agent (Ensures trip stays under $1500)
budget_planner = Agent(
    role="Budget Planner",
    goal="Find budget flights, hotels, and activities within {budget} for {destination}.",
    verbose=True,
    memory=True,
    backstory="You are a skilled budget analyst ensuring trips fit within financial constraints.",
    llm=llm,
    tools=[search_tool],
    allow_delegation=True # transfer the task to the next agent
)

# 🗺️ Itinerary Planner Agent (Creates a balanced 3-day plan)
itinerary_planner = Agent(
    role="Itinerary Planner",
    goal="Create a 3-day itinerary for {destination}, ensuring all popular sites are covered under {budget}.",
    verbose=True,
    memory=True,
    backstory="You are an expert in trip planning, ensuring travelers get the best experience within their budget.",
    llm=llm,
    tools=[search_tool],
    allow_delegation=False # As there is no next agent no need to transfer the task
)

**Step-6: Agent tasks**

In [ ]:
# 📝 Travel Research Task
research_task = Task(
    description="Find the best popular sites, weather forecast, and public transport hotels for {destination}.",
    expected_output="A list of top popular sites, a real-time weather update, and 3 hotel options near public transport.",
    tools=[search_tool],
    agent=researcher
)

# 💲 Budget Estimation Task
budget_task = Task(
    description="Find budget flights, hotel options, and daily food/transport costs for {destination}. Ensure total cost stays under {budget}.",
    expected_output="A full cost breakdown (flights, hotel, food, attractions) ensuring a $1500 budget is maintained.",
    tools=[search_tool],
    agent=budget_planner
)

# 📅 Itinerary Planning Task
itinerary_task = Task(
    description="Plan a 3-day itinerary for {destination}, focusing on popular sites, budget constraints, and real-time weather conditions.",
    expected_output="A detailed 3-day plan, considering weather and budget constraints, with transport recommendations.",
    tools=[search_tool],
    agent=itinerary_planner
)

**Step-7: Kick off the work**

In [ ]:
# 🚀 Crew Setup: All agents working together!
crew = Crew(
    agents=[researcher, budget_planner, itinerary_planner],
    tasks=[research_task, budget_task, itinerary_task],
    process=Process.sequential  # Runs tasks in sequence
)

# 🔥 Run the CrewAI Trip Advisor system for Meghalaya with a 15000 budget
result = crew.kickoff(inputs={'destination': 'Meghalaya from balasore,odisha', 'budget': '15000 INR'})
print(result)